## Подготовка данных для модели

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_csv("../data/raw/train.csv")

In [3]:
X = df.drop(columns=['Id', 'Alley', 'PoolQC', 'MiscFeature', 'SalePrice'])
y = df['SalePrice']

In [4]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=True)

In [5]:
from src.data.preprocessing import DataPreprocessor, TargetTransformer, FeatureSelector

selector = FeatureSelector().fit(X_train)

preprocessor = DataPreprocessor()
column_transformer = preprocessor.build(
    numeric_features=selector.numeric_features,
    categorical_features=selector.categorical_features
)

## Baseline модель

Сначала попробуем сделать baseline с линейной моделью с L2 регуляризацией и посмотрим на метрики. Затем также попробуем градиентный бустинг (LightGBM)

### Ridge baseline

In [25]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import Ridge
from sklearn.compose import TransformedTargetRegressor

pipeline = Pipeline(
    steps=[
        ("preprocessing", column_transformer),
        ("model", Ridge())
    ]
)

baseline_model = TransformedTargetRegressor(
    regressor=pipeline,
    func=np.log1p,
    inverse_func=np.expm1
)

In [34]:
from sklearn.metrics import root_mean_squared_log_error

baseline_model.fit(X_train, y_train)
y_pred_train = baseline_model.predict(X_train)
y_pred_val = baseline_model.predict(X_val)
print("baseline RMSE on train: %.5f" % root_mean_squared_log_error(y_train, y_pred_train))
print("baseline RMSE on val: %.5f" % root_mean_squared_log_error(y_val, y_pred_val))

baseline RMSE on train: 0.09970
baseline RMSE on val: 0.12805


In [35]:
submission = pd.read_csv("../data/raw/sample_submission.csv")
df_test = pd.read_csv("../data/raw/test.csv")

X_test = df_test.drop(columns=['Id', 'Alley', 'PoolQC', 'MiscFeature'])

y_pred_test = baseline_model.predict(X_test)
submission['SalePrice'] = y_pred_test
submission.to_csv("../data/output/my_submission.csv")

### LightGBM baseline

In [36]:
from lightgbm import LGBMRegressor

In [37]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import Ridge
from sklearn.compose import TransformedTargetRegressor

pipeline = Pipeline(
    steps=[
        ("preprocessing", column_transformer),
        ("model", LGBMRegressor())
    ]
)

baseline_model = TransformedTargetRegressor(
    regressor=pipeline,
    func=np.log1p,
    inverse_func=np.expm1
)

In [38]:
from sklearn.metrics import root_mean_squared_log_error

baseline_model.fit(X_train, y_train)
y_pred_train = baseline_model.predict(X_train)
y_pred_val = baseline_model.predict(X_val)
print("baseline RMSE on train: %.5f" % root_mean_squared_log_error(y_train, y_pred_train))
print("baseline RMSE on val: %.5f" % root_mean_squared_log_error(y_val, y_pred_val))

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001053 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3200
[LightGBM] [Info] Number of data points in the train set: 1168, number of used features: 176
[LightGBM] [Info] Start training from score 12.030658
baseline RMSE on train: 0.04227
baseline RMSE on val: 0.14054


D:\MyScripts\SDS\.venv\NASDAQ-risk-return-analisys\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
D:\MyScripts\SDS\.venv\NASDAQ-risk-return-analisys\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


In [39]:
submission = pd.read_csv("../data/raw/sample_submission.csv")
df_test = pd.read_csv("../data/raw/test.csv")

X_test = df_test.drop(columns=['Id', 'Alley', 'PoolQC', 'MiscFeature'])

y_pred_test = baseline_model.predict(X_test)
submission['SalePrice'] = y_pred_test
submission.to_csv("../data/output/my_submission.csv", index=False)

D:\MyScripts\SDS\.venv\NASDAQ-risk-return-analisys\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


В качестве базовых моделей были рассмотрены две модели: линейная регрессия Ridge и модель на основе деревьев — LightGBM.

Ridge-регрессия показала немного лучшее качество на локальной валидационной выборке (0.14054 у LightGBM и 0.12805 у Ridge), что ожидаемо ввиду её простоты и меньшей дисперсии.

В то же время LightGBM продемонстрировала сопоставимое качество на валидации и немного лучшие результаты на публичном тесте Kaggle (0.14015 у LightGBM и 0.14286 у Ridge). Учитывая способность данной модели улавливать нелинейные зависимости и взаимодействия между признаками, LightGBM была выбрана в качестве основной baseline-модели для дальнейших экспериментов.

Кроме того, бустинговые модели на основе деревьев предоставляют большую гибкость для feature engineering и подбора гиперпараметров, что делает их подходящим выбором для итеративного улучшения модели.